In [ ]:
import os
os.chdir('/mmsegmentation')

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import os

# CONFIGURATION

# Path to the JSON file
log_json = '/20250707_004059.json'  # path to the JSON file generated during training

# Folder where the file will be saved
output_dir = '/home/mmsegmentation/work_dirs/'

# Custom file name
file_name = 'training_curve.xlsx'
excel_path = os.path.join(output_dir, file_name)

# DATA READING AND MERGING

# Read the entire file
records = []
with open(log_json, 'r') as f:
    for line in f:
        records.append(json.loads(line))
df = pd.DataFrame(records)

# Filter training and validation
train_df = df[df['loss'].notna()][['step', 'loss']]
val_records = []
with open(log_json, 'r') as f:
    for line in f:
        line_data = json.loads(line)
        if 'mIoU' in line_data:
            val_records.append(line_data)
val_df = pd.DataFrame(val_records)[['step', 'mIoU']]

# Merge by 'step'
merged_df = pd.merge(train_df, val_df, on='step', how='outer').sort_values(by='step')
merged_df.rename(columns={'step': 'iter'}, inplace=True)
merged_df = merged_df[['iter', 'loss', 'mIoU']]

# SAVE FILE
merged_df.to_excel(excel_path, index=False)
print(f'✅ File saved as:\n{excel_path}')
